In [13]:
import panel as pn
import pandas as pd
from io import BytesIO
import datetime
import re

pn.extension('tabulator', css_files=[pn.io.resources.CSS_URLS['font-awesome']])

In [14]:
def process(df):
    first_dep_select = 'SGN'

    flight_dict = []
    for i in range(len(df)):
      if type(df.iloc[i]["DATE"]) == str:
        flight_dict.append(df.iloc[i].to_dict())
        flight_dict[-1]["JumpSeaters"]=[flight_dict[-1]["JumpSeaters"]]
        flight_dict[-1]["Crew"]=[flight_dict[-1]["Crew"]]
      else:
        if type(df.iloc[i]["JumpSeaters"]) == str:
          flight_dict[-1]["JumpSeaters"].append(df.iloc[i]["JumpSeaters"])
        if type(df.iloc[i]["Crew"]) == str:
          flight_dict[-1]["Crew"].append(df.iloc[i]["Crew"])

    for flight in range(len(flight_dict)):
      flight_dict[flight]["Group"] = []
      flight_dict[flight]["Positioning"] = []
      for i in range(len(flight_dict[flight]["Crew"])):
        if ("(CP+)" in flight_dict[flight]["Crew"][i] or "(CP)" in flight_dict[flight]["Crew"][i] or "(FO)" in flight_dict[flight]["Crew"][i]) and "*" not in flight_dict[flight]["Crew"][i]:
          flight_dict[flight]["Group"].append(flight_dict[flight]["Crew"][i])
      for pilot in flight_dict[flight]["Group"]:
        for flight_1 in range(len(flight_dict)):
          for i in range(len(flight_dict[flight_1]["Crew"])):
            pilotID = re.sub( r'\D' , '', pilot)
            if pilotID in flight_dict[flight_1]["Crew"][i] and "*" in flight_dict[flight_1]["Crew"][i]:
              flight_dict[flight]["Positioning"].append(pilot + "positioning on QH" + flight_dict[flight_1]["FLT"])
    
    dfa = pd.DataFrame(flight_dict)
    try:
      del dfa["Crew"]
    except:
      pass
    try:
      del dfa["Ticket Number/Remarks"]
    except:
      pass
    try:
      del dfa["Crew #"]
    except:
      pass
    try:
      del dfa["JumpSeaters"]
    except:
      pass
    std = []
    etd = []
    for i in range(len(dfa)):
      std.append(datetime.datetime(datetime.datetime.strptime(dfa.iloc[i]["DATE"], "%d/%m/%y").year, datetime.datetime.strptime(dfa.iloc[i]["DATE"], "%d/%m/%y").month, datetime.datetime.strptime(dfa.iloc[i]["DATE"], "%d/%m/%y").day, datetime.datetime.strptime(dfa.iloc[i]["STD"], "%H:%M").hour, datetime.datetime.strptime(dfa.iloc[i]["STD"], "%H:%M").minute, 00))
      try:
        etd.append(datetime.datetime(datetime.datetime.strptime(dfa.iloc[i]["DATE"], "%d/%m/%y").year, datetime.datetime.strptime(dfa.iloc[i]["DATE"], "%d/%m/%y").month, datetime.datetime.strptime(dfa.iloc[i]["DATE"], "%d/%m/%y").day, datetime.datetime.strptime(dfa.iloc[i]["ETD"], "%H:%M").hour, datetime.datetime.strptime(dfa.iloc[i]["ETD"], "%H:%M").minute, 00))
      except:
        etd.append(None)
    dfa["STD"] = pd.DataFrame(std)
    dfa["ETD"] = pd.DataFrame(etd)

    time = []

    for i in range(len(dfa)):
      if type(dfa["ETD"][i]) != pd._libs.tslibs.nattype.NaTType:
        time.append(dfa["ETD"][i])
      else:
        time.append(dfa["STD"][i])
    dfa["Time"]= time

    group_sort = []
    dfo = dfa.sort_values(by="Time")
    for i in range(len(dfo)):
      if dfo["Group"][i] not in group_sort:
        group_sort.append(dfo["Group"][i])


    sgs = []
    for group in group_sort:
      gs = []
      for i in range(len(dfa["Group"])):
        try:
          if dfa["Group"][i][0] in group or dfa["Group"][i][1] in group:
            gs.append(dfa.iloc[i])
        except:
          continue
      gs = pd.DataFrame(gs)
      if gs.iloc[0]["DEP"] == first_dep_select:
        sgs.append(gs.sort_values(by=["Time"]))

    sgs = pd.concat(sgs)
    sgs.reset_index(inplace=True)

    del sgs["index"]
    del sgs["Time"]
    gr = []

    positioning = []
    for i in range(len(sgs)):
      if "".join(sgs.iloc[i]["Positioning"]) not in positioning:
        positioning.append("".join(sgs.iloc[i]["Positioning"]))
      else:
        positioning.append(None)
    sgs["Positioning"]= positioning
    std = []
    etd = []
    for i in range(len(sgs)):
      try:
        std.append(sgs["STD"][i].strftime("%H:%M"))
      except:
        pass
      try:
        etd.append(sgs["ETD"][i].strftime("%H:%M"))
      except:
        etd.append(None)
    sgs["STD"] = pd.DataFrame(std)
    sgs["ETD"] = pd.DataFrame(etd)
    def highlight_first(s, threshold, column):
        is_nn = pd.Series(data=False, index=s.index)
        is_nn[column] = s.loc[column] != threshold
        return ['background-color: yellow' if is_nn.any() else '' for v in is_nn]

    gr = []
    for i in range(len(sgs)):
      if "".join(sgs.iloc[i]["Group"]) not in gr:
        gr.append("".join(sgs.iloc[i]["Group"]))
      else:
        gr.append(None)
    sgs["Group"] = gr

    return sgs.reset_index(drop=True).style.apply(highlight_first, threshold=None, column='Group', axis=1)

In [17]:
file_input = pn.widgets.FileInput(accept='.csv')
file_input

FileInput(accept='.csv')

In [18]:
table = pn.widgets.Tabulator(process(pd.read_csv(BytesIO(file_input.value), header=2).iloc[1:-21]), pagination='local', page_size=10)

filename, button = table.download_menu(
    text_kwargs={'name': 'Enter filename', 'value': 'output.csv'},
    button_kwargs={'name': 'Download table'}
)

pn.Row(
    pn.Column(pn.Column(file_input, filename), button),
    table
)

Row
    [0] Column
        [0] Column
            [0] FileInput(accept='.csv', filename='164788446798524.csv', mime_type='text/csv', value=b'\xef\xbb\xbfBamboo Airwa...)
            [1] TextInput(name='Enter filename', value='output.csv')
        [1] Button(name='Download table')
    [1] Tabulator(page_size=10, pagination='local', value=        DATE  ...)